# Director-AI — Protect any LLM in 10 lines

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anulum/director-ai/blob/main/notebooks/quickstart.ipynb)
[![PyPI](https://img.shields.io/pypi/v/director-ai.svg)](https://pypi.org/project/director-ai/)

**Director-AI** is a real-time LLM hallucination guardrail. It scores every LLM output
for coherence against your own knowledge base — and can halt generation mid-stream if
coherence drops below threshold.

This notebook covers:
1. Install + basic scoring
2. Custom knowledge base
3. Streaming halt demo
4. Full agent pipeline

In [ ]:
!pip install -q director-ai

## 1. Score a single response

The `CoherenceScorer` computes a dual-entropy score: logical divergence (contradiction)
and factual divergence (deviation from your facts). Both must pass.

In [ ]:
from director_ai.core import CoherenceScorer, GroundTruthStore

# Load your facts
store = GroundTruthStore()
store.add("sky color", "The sky is blue due to Rayleigh scattering.")
store.add("boiling point", "Water boils at 100 degrees Celsius at sea level.")

# Create a scorer
scorer = CoherenceScorer(threshold=0.6, ground_truth_store=store, use_nli=False)

# Check an LLM output
tests = [
    ("What color is the sky?", "The sky is blue on a clear day."),
    ("What color is the sky?", "The sky is green, obviously."),
    ("At what temperature does water boil?", "Water boils at 100 degrees Celsius."),
    ("At what temperature does water boil?", "Water boils at 50 degrees."),
]

for prompt, llm_output in tests:
    approved, score = scorer.review(prompt, llm_output)
    status = "PASS" if approved else "BLOCKED"
    print(f"[{status}] {llm_output}")
    print(f"  coherence={score.score:.3f}  h_logical={score.h_logical:.2f}  h_factual={score.h_factual:.2f}\n")

## 2. Streaming halt

The `StreamingKernel` monitors coherence **token-by-token** and halts generation
the moment it degrades. Three halt mechanisms:

- **Hard limit** — single token below absolute floor
- **Sliding window** — rolling average drops below threshold
- **Downward trend** — coherence decay over N tokens

In [ ]:
from director_ai.core import StreamingKernel

kernel = StreamingKernel(
    hard_limit=0.35,
    window_size=5,
    window_threshold=0.45,
    trend_window=4,
    trend_threshold=0.20,
)

# Simulated tokens + coherence scores (starts accurate, then drifts)
tokens = [
    "Water", " boils", " at", " 100", " C.",
    " However", " at", " high", " altitude", " it",
    " actually", " boils", " at", " only", " 50",
    " C,", " which", " means", " climbers", " can",
    " boil", " water", " with", " body", " heat", " alone.",
]
scores = [
    0.91, 0.89, 0.87, 0.90, 0.88, 0.78, 0.72,
    0.65, 0.58, 0.52, 0.46, 0.41, 0.38, 0.33,
    0.28, 0.22, 0.18, 0.15, 0.12, 0.10, 0.08,
    0.05, 0.03, 0.02, 0.01, 0.01,
]

idx = 0
def coherence_cb(tok):
    global idx
    s = scores[min(idx, len(scores) - 1)]
    idx += 1
    return s

session = kernel.stream_tokens(iter(tokens), coherence_cb)

# Print token trace
for ev in session.events:
    marker = " <<<HALT" if ev.halted else ""
    print(f"  [{ev.index:2d}] {ev.coherence:.3f}  {ev.token!r}{marker}")

print(f"\nHalted: {session.halted}")
if session.halted:
    print(f"Reason: {session.halt_reason}")
    print(f"Partial output: {session.output}")
print(f"Tokens: {session.token_count}/{len(tokens)}")
print(f"Avg coherence: {session.avg_coherence:.3f}")

## 3. Full agent pipeline

The `CoherenceAgent` orchestrates: generator → scorer → safety kernel.
It generates multiple candidates, scores each, and returns the best — or halts.

In [ ]:
from director_ai import CoherenceAgent

agent = CoherenceAgent()  # uses MockGenerator by default
result = agent.process("What color is the sky?")

print(f"Output: {result.output}")
print(f"Halted: {result.halted}")
print(f"Candidates evaluated: {result.candidates_evaluated}")
if result.coherence:
    print(f"Coherence: {result.coherence.score:.3f}")
    print(f"Approved: {result.coherence.approved}")

## 4. With a real LLM

Point the agent at any OpenAI-compatible endpoint (llama.cpp, vLLM, Ollama, etc.):

```python
agent = CoherenceAgent(llm_api_url="http://localhost:8080/completion")
result = agent.process("Explain quantum entanglement")
```

For NLI-based scoring (detects contradiction in any domain, not just pre-loaded facts):

```python
!pip install director-ai[nli]  # adds torch + transformers

scorer = CoherenceScorer(use_nli=True, threshold=0.6, ground_truth_store=store)
```

---

[GitHub](https://github.com/anulum/director-ai) |
[PyPI](https://pypi.org/project/director-ai/) |
[Docs](https://director-ai.readthedocs.io/) |
AGPL-3.0